# Running a hydrological model with multiple timeseries files

In this notebook, we show how to provide the hydrological model with multiple timeseries files. For example, one file could contain meteorological data and the other contain streamflow data, or all variables could be separate (i.e. precip, temperature, streamflow, etc.). The following instructions should make it easier to understand how to do this. for this example, we actually start from a netcdf file containing all information, and from there divide it into multiple time series netcdf files. We then use the split files to drive the model. In most user cases, different files will be provided directly by the user so no need to pre-split your files!

In [ ]:
#Cookie-cutter template necessary to provide the tools, packages and paths for the project. All notebooks
# need this template (or a slightly adjusted one depending on the required packages)
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from pathlib import Path
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import json
import netCDF4 as nc
from zipfile import ZipFile
import glob
import tempfile

# Set environment variable RAVEN_WPS_URL to "http://localhost:9099" to run on the default local server
url = os.environ.get("RAVEN_WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps")
wps = WPSClient(url)

# DATA MAIN SOURCE - DAP link to CANOPEX dataset
CANOPEX_DAP = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ets/Watersheds_5797_cfcompliant.nc'
CANOPEX_URL = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/ets/Watersheds_5797_cfcompliant.nc'

In [ ]:
# Setup some parameters to run the models. See the "canopex.ipynb" notebook for more detailed information
# on these parameters. The data we use comes from the extended CANOPEX database.
start = dt.datetime(1998, 1, 1)
stop = dt.datetime(2010, 12, 31)
watershedID = 5600

In [ ]:
# With this info, we can gather some properties from the CANOPEX database:
tmp=pd.read_csv(TESTDATA['canopex_attributes'])
basin_area=tmp['area'][watershedID]
basin_latitude = tmp['latitude'][watershedID]
basin_longitude = tmp['longitude'][watershedID]
basin_elevation = tmp['elevation'][watershedID]
basin_name=ds.watershed[watershedID].data

print("Basin name: ", basin_name)
print("Latitude: ", basin_latitude, " °N")
print("Area: ", basin_area, " km^2")

In [ ]:
# Ideally we would be able to pass the DAP link directly to Raven, but there are still some issues 
# to fix to be able to do that. For now, we'll download the series at the point of interest. 
path = Path(tempfile.mkdtemp()) /  "ts.nc"
ts = ds.isel(watershed=watershedID).sel(time=slice(start, stop))
ts.to_netcdf(path)

# Add precision on time format for Raven
D = nc.Dataset(path, mode="a")
D.variables["time"].units += " 00:00:00"
D.close()

## SECTION TO SEPARATE DISCHARGE AND MET DATA TO RECOMBINE LATER

In [ ]:
# Define the 2 new files, i.e. the meteorological data and the streamflow data
filepathMet = os.getcwd()+"/CANOPEX_Met.nc"
filepathQobs=os.getcwd()+"/CANOPEX_Qobs.nc"

# Do the extraction for the selected catchment
basindata=xr.open_dataset(CANOPEX_DAP)
newBasin=basindata.isel(watershed=watershedID)

# Generate the streamflow time-series netcdf
Qobsfile = newBasin['discharge']
Qobsfile.to_netcdf(filepathQobs)
D = nc.Dataset(filepathQobs, mode="a")
D.variables["time"].units += " 00:00:00"
D.close()

# Generate the meteorological time-series netcdf
newBasin=newBasin[['drainage_area','pr','tasmax','tasmin']]
newBasin.to_netcdf(filepathMet)
D = nc.Dataset(filepathMet, mode="a")
D.variables["time"].units += " 00:00:00"
D.close()

### Here is where we run the model with multiple input time series

In [ ]:
# The model parameters. We are forcing values here just so the model runs, the models are probably very bad choices!

params = '9.5019, 0.2774, 6.3942, 0.6884, 1.2875, 5.4134, 2.3641, 0.0973, 0.0464, 0.1998, 0.0222, -1.0919, ' \
         '2.6851, 0.3740, 1.0000, 0.4739, 0.0114, 0.0243, 0.0069, 310.7211, 916.1947'

# Model configuration parameters. Please see "canopex.ipynb" for more details. 
# This remains unchanged with multiple timeseries!
config = dict(
    start_date=start,
    end_date=stop,
    area=basin_area,
    elevation=basin_elevation,
    latitude=basin_latitude,
    longitude=basin_longitude,
    run_name='test_hmets_NRCAN',
    rain_snow_fraction='RAINSNOW_DINGMAN', 
    nc_spec=json.dumps({'tasmax': {'linear_transform': (1.0, -273.15)},'tasmin': {'linear_transform': (1.0, -273.15)},'pr': {'linear_transform': (86400.0, 0.0)}},),
)


# Here is where we must tell the model that there are multiple input files. The idea is to combine them into a list of strings,
# with each string representing a path to a netcdf file. So we could do something like this:
ts_combined=[str(filepathMet),str(filepathQobs)]
resp = wps.raven_hmets(ts=ts_combined, params=params, **config)

# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy the locally. 
[hydrograph, storage, solution, diagnostics, rv] = resp.get(asobj=True)
print(diagnostics)

### You can even invert the order of the netcdf files. Raven will detect which files contain which variables, so the order is not important!

In [ ]:
# Test with reversed timeseries files:
ts_combined=[str(filepathQobs),str(filepathMet)]
resp = wps.raven_hmets(ts=ts_combined, params=params, **config)

# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy the locally. 
[hydrograph, storage, solution, diagnostics, rv] = resp.get(asobj=True)
print(diagnostics)

### As you can see, the NSE values and RMSE values are identical. You can pass as many NetCDF files as you have variables in any order and it will still work.